In [1]:
import time

import numpy as np
import scipy as sc

import bsplines  as bsp

import HyCho_FEM as fem
import HyCho_PIC
#import HyCho_PIC as pic

import utilitis_opt as utils_opt
import utilitis_pic_Rel
#import utilitis_pic_Rel as utils_pic_fast

import matplotlib.pyplot as plt

from scipy.sparse.linalg import splu
from scipy.sparse        import block_diag
from scipy.sparse.linalg import inv

In [2]:
#====================================================================================
#  calling epyccel
#====================================================================================
from pyccel.epyccel import epyccel
utils_pic_fast = epyccel(utilitis_pic_Rel)

print('pyccelization of pic functions done!')
#====================================================================================


#====================================================================================
#  calling epyccel
#====================================================================================
from pyccel.epyccel import epyccel
pic = epyccel(HyCho_PIC)

print('pyccelization of pic functions done!')
#====================================================================================

pyccelization of pic functions done!
pyccelization of pic functions done!


In [3]:
#===== Is this run a restart? (restart = 0: no, restart = 1: yes) ===================
restart            = 0

max_time           = 30*60         # maximum runtime of program in minutes
time_restart_files = 40*60          # time interval for restart files in minutes

name_particles     = 'restart_files/particles1.npy'
name_fields        = 'restart_files/fields1.npy'
name_time_step     = 'restart_files/time_step1.npy'
name_control       = 'restart_files/control_variate1.npy'
#====================================================================================


#=========================== time integration =======================================
time_integr = 1                    # do time integration? (1 : yes, 0: no)
title       = 'results/run_L=327.7_Nel=2000_T=300_dt=0.04_Np=4e6_nuh=6e-2_xi=0_False.txt'           # name of file to save data
#====================================================================================


#===== physical parameters ==========================================================
wpe   = 5.                         # cold electron plasma frequency
nuh   = 6e-2                       # ratio of cold/hot electron densities (nh/nc)
nh    = nuh*wpe**2                 # hot electron density
wpar  = 0.2                        # parallel thermal velocity of energetic particles
wperp = 0.53                       # perpendicular thermal velocity of energetic particles
xi    = 0.                         # inhomogeneity factor of background magnetic field

rel   = 1                          # relativistic effects? (1: yes, 0: no)
bc_d  = 1                          # damping of E and j at boundaries? (1: yes, 0: no)
bc_f  = 1                          # field line dependence of initial distribution function? (1: yes, 0: no)
#===================================================================================



#===== numerical parameters =========================================================
bc      = False                    # boundary conditions (True: periodic, False: homogeneous Dirichlet)
k       = 2.                       # wavenumber of initial wave field perturbations
Lz      = 327.7                     # length of z-domain
Nel     = 1800                      # number of elements z-direction
T       = 1.                     # simulation time
dt      = 0.04                      # time step
p       = 3                        # degree of B-spline basis functions in V0
Np      = np.int(5e6)              # number of markers
control = 1                        # control variate for noise reduction? (1: yes, 0: no)
Ld      = 0.046*Lz                 # length of damping region at each end
#====================================================================================


#===== evaluation points for the magnetic field======================================
eva_points_Bx = np.array([100., 200., 300.])
#====================================================================================


#===== initial conditions ===========================================================
amp = 1e-4                         # amplitude of initial wave field perturbations

Ex0 = lambda z : 0*z               # initial Ex
Ey0 = lambda z : 0*z               # initial Ey

'''
def Bx0(z):
    
    values = np.zeros(z.shape, dtype=float)
    modesz = np.linspace(0, Nel, Nel + 1) - Nel/2
    modesz = np.delete(modesz, int(Nel/2))
    
    for i in range(Nel):
        values += amp*np.sin(2*np.pi*modesz[i]*z/Lz)
        
    return values
'''

Bx0 = lambda z : amp*np.sin(104*2*np.pi*z/Lz) # initial Bx
By0 = lambda z : 0*z               # initial By
jx0 = lambda z : 0*z               # initial jcx
jy0 = lambda z : 0*z               # initial jcy
#====================================================================================



#===== discretization of spatial domain =============================================
dz   = Lz/Nel                                # element size
el_b = np.linspace(0., Lz, Nel + 1)          # element boundaries

Nbase0     = Nel + p - bc*p                  # total number of basis functions in V0
Nbase0_dof = Nbase0 - 2 + 2*bc               # number of degrees of freedom in V1

Nbase1     = Nbase0 - 1 + bc                 # total number of basis functions in V1
Nbase1_dof = Nbase1                          # number of degrees of freedom in V1
#====================================================================================



#===== some diagnostic values =======================================================
nh       = nuh*wpe**2                                 # hot electron density
Eh_eq    = Lz*nh/2*(wpar**2 + 2*wperp**2)             # equilibrium energetic electron energy

energies = np.empty(4, dtype=float)                   # energies: E, B, Cold, Hot

Bx       = np.empty(len(eva_points_Bx), dtype=float)  
#====================================================================================


#===== background field in z-direction ==============================================
B_background_z = lambda z : 1. + xi*(z - Lz/2)**2
#====================================================================================


#===== initial energetic electron distribution function =============================
def fh0(z, vx, vy, vz):

    xiB = 1. - 1/B_background_z(z)
    xiz = 1. + (wperp**2/wpar**2 - 1.)*xiB*bc_f

    return nh/((2*np.pi)**(3/2)*wpar*wperp**2)*np.exp(-vz**2/(2*wpar**2) - xiz*(vx**2 + vy**2)/(2*wperp**2))
#====================================================================================


#===== Maxwellian for control variate ===============================================
maxwell = lambda vx, vy, vz : nh/((2*np.pi)**(3/2)*wpar*wperp**2)*np.exp(-vz**2/(2*wpar**2) - (vx**2 + vy**2)/(2*wperp**2))
#====================================================================================


#===== sampling distribution for initial markers ====================================
g_sampling = lambda vx, vy, vz : 1/((2*np.pi)**(3/2)*wpar*wperp**2)*np.exp(-vz**2/(2*wpar**2) - (vx**2 + vy**2)/(2*wperp**2))*1/Lz
#====================================================================================



#===== masking function to damp wave fields near boundaries =========================
def damp(z):

    if z <= Ld:
        return np.sin(np.pi*z/(2*Ld))
    elif z >= Lz - Ld:
        return np.sin(np.pi*(Lz - z)/(2*Ld))
    else:
        return 1
#====================================================================================



#===== spline knot vector, global mass matrices (in V0 and V1) and gradient matrix ==
T0 = bsp.make_knots(el_b, p, bc)
T1 = T0[1:-1]

M0 = fem.mass_V0(T0, p, bc)
M1 = fem.mass_V1(T0, p, bc)
MB = fem.mass_V0_B(T0, p, bc, B_background_z)

G  = sc.sparse.csc_matrix(fem.GRAD(T0, p, bc))

if bc == False:
    M0 = M0[1:-1, 1:-1]
    MB = MB[1:-1, 1:-1]
    
    G  = G[:, 1:-1]

    
D = sc.sparse.csr_matrix(bsp.collocation_matrix(T1, p - 1, eva_points_Bx, bc, normalize=True))
print('matrix assembly done!')
#====================================================================================



#=================== coefficients for pp-forms ======================================
if p == 3:
    pp_0 = np.asfortranarray([[1/6, -1/(2*dz), 1/(2*dz**2), -1/(6*dz**3)], [2/3, 0., -1/dz**2, 1/(2*dz**3)], [1/6, 1/(2*dz), 1/(2*dz**2), -1/(2*dz**3)], [0., 0., 0., 1/(6*dz**3)]])
    pp_1 = np.asfortranarray([[1/2, -1/dz, 1/(2*dz**2)], [1/2, 1/dz, -1/dz**2], [0., 0., 1/(2*dz**2)]])
elif p == 2:
    pp_0 = np.asfortranarray([[1/2, -1/dz, 1/(2*dz**2)], [1/2, 1/dz, -1/dz**2], [0., 0., 1/(2*dz**2)]])
    pp_1 = np.asfortranarray([[1., -1/dz], [0., 1/dz]])
else:
    print('Only cubic and quadratic splines implemented!')
#====================================================================================



#===== reserve memory for unknowns ==================================================
ex     = np.empty(Nbase0, dtype=float)
ey     = np.empty(Nbase0, dtype=float)
bx     = np.empty(Nbase1, dtype=float)
by     = np.empty(Nbase1, dtype=float)
yx     = np.empty(Nbase0, dtype=float)
yy     = np.empty(Nbase0, dtype=float)

uj     = np.empty(4*Nbase0_dof + 2*Nbase1_dof, dtype=float)
z_old  = np.empty(Np, dtype=float)
spans0 = np.empty(Np, dtype=int)

jh_x   = np.empty(Nbase0, dtype=float)
jh_y   = np.empty(Nbase0, dtype=float)

Fh     = np.zeros(4*Nbase0_dof + 2*Nbase1_dof, dtype=float)
#====================================================================================



#===== initial coefficients with commuting projectors ===============================
proj = fem.projectors_1d(T0, p, bc)

ex[:] = proj.PI_0(Ex0)
ey[:] = proj.PI_0(Ey0)
bx[:] = proj.PI_1(Bx0)
by[:] = proj.PI_1(By0)
yx[:] = proj.PI_0(jx0)
yy[:] = proj.PI_0(jy0)

if bc == False:
    uj[:] = np.concatenate((ex[1:-1], ey[1:-1], bx, by, yx[1:-1], yy[1:-1]))
    
else:
    uj[:] = np.concatenate((ex, ey, bx, by, yx, yy))

print('projection of initial fields done!')
#====================================================================================



#===== construct block matrices for field update ====================================
I  = sc.sparse.identity(Nbase1_dof)
A1 = sc.sparse.bmat([[M0, None, None, None, None, None],[None, M0, None, None, None, None], [None, None, I, None, None, None], [None, None, None, I, None, None], [None, None, None, None, M0, None], [None, None, None, None, None, M0]], format='csc')
A2 = sc.sparse.bmat([[None, None, None, G.T.dot(M1), -M0, None],[None, None, -G.T.dot(M1), None, None, -M0], [None, G, None, None, None, None], [-G, None, None, None, None, None], [wpe**2*M0, None, None, None, None, -MB], [None, wpe**2*M0, None, None, MB, None]], format='csc')

LHS = A1 - dt/2*A2
RHS = A1 + dt/2*A2

LU = sc.sparse.linalg.splu(LHS)
print('LU factorization done!')


if bc_d == 1:
    
    if bc == False:
        greville = bsp.greville(T0, p, bc)[1:-1]
        colloq = sc.sparse.csc_matrix(bsp.collocation_matrix(T0, p, greville, bc)[:, 1:-1])
    else:
        greville = bsp.greville(T0, p, bc)
        colloq = sc.sparse.csc_matrix(bsp.collocation_matrix(T0, p, greville, bc))
        
    g_greville = np.zeros(Nbase0_dof, dtype=float)
    
    for i in range(Nbase0_dof):
        g_greville[i] = damp(greville[i])
            
    G_greville = sc.sparse.diags(g_greville, 0)
    DAMP       = inv(colloq).dot(G_greville.dot(colloq))
             
else:
    DAMP = sc.sparse.identity(Nbase0_dof)
    
DAMP_block = sc.sparse.block_diag((DAMP, DAMP, sc.sparse.identity(Nbase1_dof), sc.sparse.identity(Nbase1_dof), DAMP, DAMP), format='csr')
print('damping assembly done!')
#====================================================================================

matrix assembly done!
projection of initial fields done!
LU factorization done!
damping assembly done!


In [20]:
#===== Is this run a restart? (restart = 0: no, restart = 1: yes) ===================
restart            = 0

max_time           = 30*60          # maximum runtime in minutes
time_restart_files = 40*60          # time after which the current configuration is saved in minutes

name_particles     = 'restart_files/particles1.npy'
name_fields        = 'restart_files/fields1.npy'
name_time_step     = 'restart_files/time_step1.npy'
name_control       = 'restart_files/control_variate1.npy'
#====================================================================================



#===== saving data? (save = 1: yes, save = 0: no). If yes, name directory ===========
save        = 1
title       = 'results/short_old.txt' 
saving_step = 1                    # save data only every saving_stepth time step
time_integr = 1                    # do time integration? (1 : yes, 0: no)
#====================================================================================



#===== physical parameters ==========================================================
eps0  = 1.0                        # vacuum permittivity
mu0   = 1.0                        # vacuum permeability
c     = 1.0                        # speed of light
qe    = -1.0                       # electron charge
me    = 1.0                        # electron mass
B0z   = 1.0                        # minimum of background magnetic field in z-direction
wce   = qe*B0z/me                  # electron cyclotron frequency
wpe   = 5*np.abs(wce)              # cold electron plasma frequency
nuh   = 6e-2                       # ratio of cold/hot electron densities (nh/nc)
nh    = nuh*wpe**2                 # hot electron density
wpar  = 0.2*c                      # parallel thermal velocity of energetic particles
wperp = 0.53*c                     # perpendicular thermal velocity of energetic particles

xi    = 0.                         # inhomogeneity factor of background magnetic field

bcs_d = 1                          # damping of wave fields at boundaries? (1: yes, 0: no)
bcs_g = 1                          # field line dependence of initial distribution function? (1: yes, 0: no)
#====================================================================================



#===== initial conditions ===========================================================
k   = 2.                           # wavenumber of initial wave field perturbations
amp = 1e-4                         # amplitude of initial wave field perturbations
eps = 0.                           # amplitude of spatial pertubation of initial distribution function 


Ex0 = lambda z : 0*z               # initial Ex
Ey0 = lambda z : 0*z               # initial Ey
Bx0 = lambda z : amp*np.sin(104*2*np.pi*z/Lz) # initial Bx
By0 = lambda z : 0*z               # initial By
jx0 = lambda z : 0*z               # initial jcx
jy0 = lambda z : 0*z               # initial jcy
#====================================================================================



#===== numerical parameters =========================================================
Lz      = 327.7                    # length of z-domain
Nel     = 1800                     # number of elements z-direction
T       = 1.                      # simulation time
dt      = 0.04                     # time step
p       = 3                        # degree of B-spline basis functions in V0
Np      = np.int(5e6)              # number of markers
control = 1                        # control variate for noise reduction? (1: yes, 0: no)

Ld      = 0.046*Lz                 # length of damping region at each end
#====================================================================================


#===== evaluation points for the magnetic field======================================
#eva_points_Bx = np.linspace(40., 280., 7)
eva_points_Bx = np.array([100., 200., 300.])
#====================================================================================



#====== create parameter list =======================================================
pa = np.zeros(1*(Nel + p - 1) + 5)

pa[0]  = eps0
pa[1]  = mu0
pa[2]  = c
pa[3]  = qe 
pa[4]  = me 
pa[5]  = B0z 
pa[6]  = wce 
pa[7]  = wpe 
pa[8]  = nuh 
pa[9]  = nh 
pa[10] = wpar 
pa[11] = wperp 
pa[12] = k  
pa[13] = amp 
pa[14] = eps 
pa[15] = Lz 
pa[16] = Nel 
pa[17] = T 
pa[18] = dt 
pa[19] = p 
pa[20] = Np
pa[21] = control
pa[22] = saving_step

pa[23] = xi
pa[24] = Ld

pa[29] = bcs_d
pa[30] = bcs_g
#====================================================================================



#===== discretization of spatial domain =============================================
dz   = Lz/Nel                                # element size
el_b = np.linspace(0, Lz, Nel + 1)           # element boundaries

Nbase0   = Nel + p                           # total number of basis functions in V0
Nbase0_0 = Nbase0 - 2                        # number of degrees of freedom in V1

Nbase1   = Nbase0 - 1                        # total number of basis functions in V1
Nbase1_0 = Nbase1                            # number of degrees of freedom in V1
#====================================================================================



#===== some diagnostic values =======================================================
Eh_eq = Lz*nh*me/2*(wpar**2 + 2*wperp**2)    # equilibrium energetic electron energy

en_E = np.array([])                          # electric field energy
en_B = np.array([])                          # magnetic field energy
en_C = np.array([])                          # cold plasma energy
en_H = np.array([])                          # energetic electron energy
#====================================================================================



#===== background field in z-direction ==============================================
B_background_z = lambda z : B0z*(1 + xi*(z - Lz/2)**2)
#====================================================================================



#===== initial energetic electron distribution function =============================
def fh0(z, vx, vy, vz):

    xiB = 1 - B0z/B_background_z(z)
    xiz = 1 + (wperp**2/wpar**2 - 1)*xiB*bcs_g

    return (1 + eps*np.cos(k*z))*nh/((2*np.pi)**(3/2)*wpar*wperp**2)*np.exp(-vz**2/(2*wpar**2) - xiz*(vx**2 + vy**2)/(2*wperp**2))
#====================================================================================



#===== Maxwellian for control variate ===============================================
maxwell = lambda vx, vy, vz : nh/((2*np.pi)**(3/2)*wpar*wperp**2)*np.exp(-vz**2/(2*wpar**2) - (vx**2 + vy**2)/(2*wperp**2))
#====================================================================================


#===== sampling distribution for initial markers ====================================
g_sampling = lambda vx, vy, vz : 1/((2*np.pi)**(3/2)*wpar*wperp**2)*np.exp(-vz**2/(2*wpar**2) - (vx**2 + vy**2)/(2*wperp**2))*1/Lz
#====================================================================================



#===== masking function to damp wave fields near boundaries =========================
def damp(z):

    if z <= Ld:
        return np.sin(np.pi*z/(2*Ld))
    elif z >= Lz - Ld:
        return np.sin(np.pi*(Lz - z)/(2*Ld))
    else:
        return 1.0
#====================================================================================



#===== spline knot vector, global mass matrices (in V0 and V1) and gradient matrix ==
Tz = bsp.make_knots(el_b, p, False)
tz = Tz[1:-1]

M0, C0 = utils_opt.matrixAssembly_V0(p, Nbase0, Tz, False)
M1 = utils_opt.matrixAssembly_V1(p, Nbase0, Tz, False)
Mb = utils_opt.matrixAssembly_backgroundField(p, Nbase0, Tz, False, B_background_z)

G = utils_opt.GRAD_1d(p, Nbase0, False)

D = bsp.collocation_matrix(tz, p - 1, eva_points_Bx, False, normalize=True)

print('matrix assembly done!')
#====================================================================================



#===== reserve memory for unknowns ==================================================
ex = np.empty(Nbase0)
ey = np.empty(Nbase0)
bx = np.empty(Nbase1)
by = np.empty(Nbase1)
yx = np.empty(Nbase0)
yy = np.empty(Nbase0)

uj = np.empty(4*Nbase0_0 + 2*Nbase1_0)

z_old = np.empty(Np)
#====================================================================================



#===== initial coefficients with commuting projectors ===============================
proj = utils_opt.projectors_1d(p, Nbase0, Tz, False)

ex[:] = proj.PI_0(Ex0)
ey[:] = proj.PI_0(Ey0)
bx[:] = proj.PI_1(Bx0)
by[:] = proj.PI_1(By0)
yx[:] = proj.PI_0(jx0)
yy[:] = proj.PI_0(jy0)

uj[:] = np.concatenate((ex[1:-1], ey[1:-1], bx, by, yx[1:-1], yy[1:-1]))

print('projection of initial fields done!')
#====================================================================================




#===== construct block matrices for field update ====================================
ZERO_00 = np.zeros((Nbase0_0, Nbase0_0))
ZERO_01 = np.zeros((Nbase0_0, Nbase1_0))
ZERO_11 = np.zeros((Nbase1_0, Nbase1_0))

A1 = np.diag(np.ones(4*Nbase0_0 + 2*Nbase1_0))

A1[0:Nbase0_0, 0:Nbase0_0] = M0
A1[Nbase0_0:2*Nbase0_0, Nbase0_0:2*Nbase0_0] = M0

A1[2*Nbase0_0 + 2*Nbase1_0:3*Nbase0_0 + 2*Nbase1_0, 2*Nbase0_0 + 2*Nbase1_0:3*Nbase0_0 + 2*Nbase1_0] = M0
A1[3*Nbase0_0 + 2*Nbase1_0:4*Nbase0_0 + 2*Nbase1_0, 3*Nbase0_0 + 2*Nbase1_0:4*Nbase0_0 + 2*Nbase1_0] = M0

A2 = np.block([[ZERO_00, ZERO_00, ZERO_01, c**2*np.dot(G.T, M1), -mu0*c**2*M0, ZERO_00], [ZERO_00, ZERO_00, -c**2*np.dot(G.T, M1), ZERO_01, ZERO_00, -mu0*c**2*M0], [ZERO_01.T, G, ZERO_11, ZERO_11, ZERO_01.T, ZERO_01.T], [-G, ZERO_01.T, ZERO_11, ZERO_11, ZERO_01.T, ZERO_01.T], [eps0*wpe**2*M0, ZERO_00, ZERO_01, ZERO_01, ZERO_00, qe/me*Mb], [ZERO_00, eps0*wpe**2*M0, ZERO_01, ZERO_01, -qe/me*Mb, ZERO_00]])

LHS = sc.sparse.csc_matrix(A1 - 1/2*dt*A2)
RHS = sc.sparse.csc_matrix(A1 + 1/2*dt*A2)

LU = sc.sparse.linalg.splu(LHS)

print('LU factorization done!')


if bcs_d == 1:
    grev = bsp.greville(Tz, p, False)
    coll = bsp.collocation_matrix(Tz, p, grev, False)[1:-1, 1:-1]
    gi = np.zeros(Nbase0)

    for i in range(Nbase0):
        gi[i] = damp(grev[i])

    Gi = np.diag(gi[1:-1])

    DAMP = np.dot(np.dot(np.linalg.inv(coll), Gi), coll)
else:
    DAMP = np.identity(Nbase0_0)
    
DAMP_block = sc.linalg.block_diag(DAMP, DAMP, np.identity(Nbase1_0), np.identity(Nbase1_0), DAMP, DAMP)
    
print('damping assembly done!')
#====================================================================================

matrix assembly done!
projection of initial fields done!
LU factorization done!
damping assembly done!


In [5]:
np.allclose(RHS_old.toarray(), RHS.toarray())

True

In [6]:
np.allclose(LHS_old.toarray(), LHS.toarray())

True

In [7]:
np.allclose(DAMP_block_old, DAMP_block.toarray())

True

In [52]:
#===== create particles (z, vx, vy, vz, wk) and sample according to sampling distribution
particles       = np.zeros((Np, 5), order='F', dtype=float)

particles[:, 0] = np.random.rand (Np)*Lz
particles[:, 1] = np.random.randn(Np)*wperp
particles[:, 2] = np.random.randn(Np)*wperp
particles[:, 3] = np.random.randn(Np)*wpar
particles[:, 4] = np.random.rand(Np)

np.save('particles', particles)
#====================================================================================

In [21]:
particles = np.load('particles.npy')
z_old = np.empty(Np)

jh = np.zeros(2*Nbase0)
Fh = np.zeros(4*Nbase0_0 + 2*Nbase1_0)


g0 = g_sampling(particles[:, 1], particles[:, 2], particles[:, 3])
w0 = fh0(particles[:, 0], particles[:, 1], particles[:, 2], particles[:, 3])/g_sampling(particles[:, 1], particles[:, 2], particles[:, 3])

In [22]:
z_old[:] = particles[:, 0]
utils_pic_fast.borisGemRel_bc_2(particles, -dt/2, qe, me, Tz, tz, p, ex, ey, bx, by, B0z, xi, Lz, c)

particles[:, 0] = z_old
particles[:, 4] = w0 - control*maxwell(particles[:, 1], particles[:, 2], particles[:, 3])/g0

me/(2*Np) * particles[:, 4].dot(particles[:, 1]**2 + particles[:, 2]**2 + particles[:, 3]**2) + control*Eh_eq

147.90739484884202

In [16]:
particles[:10, 1]

array([ 0.74188978, -0.5786941 , -0.03251462, -0.32875401, -0.53128939,
       -0.46836663,  0.40065219,  0.29878744,  1.30271792, -0.12234293])

In [5]:
pic.current(particles[:, 0], particles[:, 1:], T0, p, np.floor(particles[:, 0]/dz).astype(int) + p, jh_x, jh_y, Nbase0, rel)

NameError: name 'T0' is not defined

In [11]:
jh = np.empty(2*Nbase0, dtype=float)
utils_pic_fast.hotCurrentRel_bc_2(particles[:, 0], particles[:, 1:], T0, p, -1., jh, 1.)

In [12]:
np.allclose(jh[0::2], jh_x)

True

In [13]:
np.allclose(jh[1::2], jh_y)

True

In [6]:
#particles_hycho = np.copy(particles)
particles_old   = np.copy(particles)

In [7]:
bx

array([-3.59042551e-09,  2.23260154e-06,  9.58544172e-06, ...,
       -9.58544172e-06, -2.23260154e-06,  3.59042551e-09])

In [14]:
ex     = np.random.rand(Nbase0)
ey     = np.random.rand(Nbase0)
bx     = np.random.rand(Nbase1)
by     = np.random.rand(Nbase1)

In [39]:
pic.pusher_reflecting(particles_hycho, -dt/2, T0, T1, p, np.floor(particles_hycho[:, 0]/dz).astype(int) + p, Lz, dz, ex, ey, bx, by, pp_0, pp_1, xi, rel)

In [8]:
utils_pic_fast.borisGemRel_bc_2(particles_old, -dt/2, qe, me, Tz, tz, p, ex, ey, bx, by, B0z, xi, Lz, c)

In [48]:
np.allclose(particles_hycho, particles_old, atol=1e-14, rtol=1e-8)

True

In [43]:
particles_hycho[:10, 3]

array([-0.11215602, -0.10063501, -0.24428482,  0.04063113, -0.02269177,
       -0.06058713,  0.39993321, -0.27301069,  0.16482462,  0.09644344])

In [57]:
particles_old[:10, 3]

array([-0.11215602, -0.10063501, -0.24428482,  0.04063113, -0.02269177,
       -0.06058713,  0.39993321, -0.27301069,  0.16482462,  0.09644344])

In [9]:
particles_old[:, 4] = w0 - control*maxwell(particles_old[:, 1], particles_old[:, 2], particles_old[:, 3])/g0

In [10]:
particles_old[:, 4]

array([-1.97970038e-04,  4.87099696e-05,  1.79928030e-05, ...,
        1.54546770e-03,  1.45301404e-04,  4.59360706e-04])

In [11]:
me/(2*Np) * particles_old[:, 4].dot(particles_old[:, 1]**2 + particles_old[:, 2]**2 + particles_old[:, 3]**2) + control*Eh_eq 

147.90739484884202

In [19]:
np.save('particles_old', particles)